In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import unidecode

In [2]:
df2 = pd.read_csv('BasePareceresFinal.csv')

C:\Users\bruno\AppData\Local\Temp\ipykernel_23280\1711057017.py:1: DtypeWarning: Columns (10,11,12,13,20,28,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('BasePareceresFinal.csv')


In [4]:
df2.head(30).to_excel('exemplo_base.xlsx')

In [4]:
df2 = df2.drop(axis=1, columns=['Unnamed: 0', 'nota_id', 'num_notas',
                              'status', 'paciente_cidade', 'advogado_nome', 'advogado_numOAB', 'processo_vara',
                              'diagnostico_confirmacao', 'tecnologia_tipo', 'tecnologia_comercial',
                              'tecnologia_viaAdministracao', 'tecnologia_posologia', 'outrasTec_tecnologia',
                              'outrasTec_opcoesSUS', 'outrasTec_opcoesGenSim',
                              'custos_tecnologia', 'custos_lab', 'custos_marca',
                              'custos_apresentacao', 'custos_precoFabrica', 'custos_precoGov',
                              'custos_precoCons', 'custosMes_tecnologia', 'custosMes_doseDiaria',
                              'custosMes_precoGov', 'custosMes_precoCon', 'custosMes_fonte',
                              'evidencias_tecnologia', 'evidencias_eficaciaSeguranca',
                              'evidencias_beneficio', 'conclusao_tecnologia', 'conclusao_motivacao',
                              'conclusao_refs', 'conclusao_outrasInfos', 'conclusao_anexo1',
                              'conclusao_anexo2', 'conclusao_anexo3', 'conclusao_anexo4'])



In [5]:
df2['cid_simplificado'] = df2['diagnostico_cid'].str.extract(r'([A-Z]\d{2})')

In [6]:
# Seleciono casos relevantes
lista = df2.loc[df2['instResponsavel'] != 'Não preenchido'].instResponsavel.value_counts().head(24).index.tolist()
df = df2.loc[df2['instResponsavel'].isin(lista)]

In [7]:
df['tecnologia_principio'] = df['tecnologia_principio'].astype(str).str.lower() \
                                                    .str.replace(' ', '') \
                                                    .apply(unidecode.unidecode)
df = df[df['conclusao_resultado'].isin(['Favorável', 'Não favorável'])]

C:\Users\bruno\AppData\Local\Temp\ipykernel_11300\3561700512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tecnologia_principio'] = df['tecnologia_principio'].astype(str).str.lower() \


In [8]:
# Supondo que df seja seu DataFrame original
# Passo 1: Contar as combinações de 'cid_simplificado' e 'tecnologia_principio'
contagem_combinacoes = df.groupby(['cid_simplificado', 'tecnologia_principio']).size().reset_index(name='counts')

# Passo 2: Filtrar as combinações que aparecem mais de 10 vezes
combinacoes_filtradas = contagem_combinacoes[contagem_combinacoes['counts'] >= 10]

# Passo 3: Mesclar com o DataFrame original para obter as linhas completas
df_controle = df.merge(combinacoes_filtradas[['cid_simplificado', 'tecnologia_principio']], on=['cid_simplificado', 'tecnologia_principio'])

In [9]:
resultados = []

# Lista de instituições e princípios ativos
lista_instituicoes = df_controle['instResponsavel'].unique()

for inst in lista_instituicoes:
    print(inst)

    lista_meds = df_controle.loc[df_controle['instResponsavel'] == inst]['tecnologia_principio'].unique()

    for med in lista_meds:
        print(med)

        lista_cids = df_controle.loc[(df_controle['instResponsavel'] == inst) & (df_controle['tecnologia_principio'] == med)]['cid_simplificado'].unique()
        
        for cid in lista_cids:
            print(cid)
            # Filtra os dados em df_controle
            filtro = df_controle.loc[
                (df_controle['instResponsavel'] == inst) & 
                (df_controle['tecnologia_principio'] == med) & 
                (df_controle['cid_simplificado'] == cid)
            ]
            qtd_total = len(filtro)

            # Verifica se o número de registros é maior que 10
            if qtd_total > 10:
                conclusao_stats = filtro['conclusao_resultado'].value_counts(normalize=True).round(3) * 100

                # Garante que as estatísticas estão formatadas corretamente
                if not conclusao_stats.empty:
                    resultados.append({
                        'Órgão de ATS': inst,
                        'Tecnologia': med,
                        'CID': cid,
                        'Nº de pareceres': qtd_total,
                        'Conclusoes': conclusao_stats.to_dict()
                    })
                else:
                    print(f"⚠️ Estatísticas de 'conclusao' vazias para {inst}, {med}, {cid}")

# Converte a lista de resultados em um DataFrame
df_resultados2 = pd.DataFrame(resultados)

# Expande a coluna 'Conclusoes' apenas se ela existir
if 'Conclusoes' in df_resultados2.columns:
    df_resultados2 = df_resultados2.join(pd.DataFrame(df_resultados2['Conclusoes'].tolist()))

    # Preenche valores NaN com 0 (onde a categoria não foi registrada)
    df_resultados2.fillna(0, inplace=True)

    # Remove a coluna 'Conclusoes'
    df_resultados2.drop(columns=['Conclusoes'], inplace=True)

# Salva o DataFrame em um arquivo Excel
df_resultados2.to_excel('controle_por_tratamento_e_por_CID.xlsx', index=False)


NatJus RN
pembrolizumabe
C43
C67
C16
C64
C53
C32
C81
rivaroxabana
I48
I26
I64
I34
I70
I49
I82
I50
I69
I42
I25
aspartatodeornitina
K70
eltrombopagueolamina
D61
D69
rituximabe
D69
C91
N04
G36
C82
G61
C85
denosumabe
M80
M81
M82
D16
C41
dupilumabe
L20
levetiracetam
G40
ranibizumabe
H35
H34
H36
brometodetiotropio
J44
everolimo
C17
C25
C64
palmitatodepaliperidona
F20
F25
lenalidomida
C90
abiraterona
C61
nitisinona
E70
bortezomibe
C90
bevacizumabe
H35
C18
H34
C20
H36
C53
C22
C71
empagliflozina
E11
I50
I25
vedolizumabe
K50
K51
omalizumabe
J45
L50
prucaloprida
K59
esilatodenintedanibe
J84
vismodegibe
C44
micofenolatodemofetila
N04
M34
J84
nintedanibe
J84
nivolumabe
C43
C64
apixabana
I48
I64
acetatodeleuprorrelina
E30
E22
lisdexanfetamina
F90
aflibercepte
H35
H36
H34
cloridratodemetilfenidato
F90
F84
F71
acetatodeabiraterona
C61
riociguate
I27
acetatodegosserrelina
C50
N80
acidozoledronico
M81
canabidiol
R52
G40
F84
leflunomida
M05
galcanezumabe
G43
regorafenibe
C18
C20
teriparatida
M80
M81
lira